# Curso Teste A/B/n - PA James

## 0.1. Imports

In [1]:
import pandas as pd
import numpy as np

from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.stats.power import GofChisquarePower, TTestIndPower

# 1.0. Carregar Dados

In [2]:
d = { 'variant': ['interact', 'connect', 'learn', 'help', 'services'],
      'visits': [10283, 2842, 2747, 3180, 2064],
      'clicks_all': [3714, 1587, 1652, 1717, 1348],
      'clicks_link': [42, 53, 21, 38, 45]}

data = pd.DataFrame(d)
data['conversion'] = data['clicks_link'] / data['clicks_all']
data

,variant,visits,clicks_all,clicks_link,conversion
0,interact,10283,3714,42,0.011309
1,connect,2842,1587,53,0.033396
2,learn,2747,1652,21,0.012712
3,help,3180,1717,38,0.022132
4,services,2064,1348,45,0.033383


# 2.0. Design de Experimentos

## 2.1. Formulação das Hipóteses

In [3]:
# H0: Não á nenhuma diferença entre o CTR das variantes da página
# H1: Há uma diferença entre os CTR das variantes da página
#
# Teste Bi-Caudal

## 2.2. Parâmetros do Experimento

In [4]:
k = len(data['clicks_all'])
actual_dist = data['clicks_link'] / data['clicks_link'].sum()
expected_dist = [1/k]*k

In [5]:
efect_size = chisquare_effectsize(expected_dist, actual_dist)
alpha = 0.05
power = 0.80

## Goodness of fitness
# sample_size = GofChisquarePower().solve_power(
#     effect_size=efect_size,
#     alpha=alpha,
#     power=power,
#     n_bins=(2-1)*(5-1) # (nFeature-1)*(nRows-1)
# )

# Teste de independência (Homogeneidade)
sample_size = TTestIndPower().solve_power(
    effect_size=efect_size,
    alpha=alpha,
    power=power
)

sample_size = np.ceil(sample_size).astype(int)

print(f'Tamanho mínimo da amostra por variante: {sample_size}')
print(f'Tamanho total amostras: {k*sample_size}')

Tamanho mínimo da amostra por variante: 222
Tamanho total amostras: 1110


# 3.0. Conversão da página em Faturamento